In [2]:
#Code to create 3 150*150 images by taking optical flow from 2 alternate frames
import os
import numpy as np
import cv2 as cv
import random
from cv2.optflow import calcOpticalFlowSF
from PIL import Image
# import IPython.display as display
from prepare_dataset import *
import glob

In [66]:
#Function to generate an image array when centre pixels are provided
def create_image(frame,i,j):
    if i-75>=0:
        if i+75<=1080-1:
            if j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[i-75:i+75,j-75:j+75,0:3]
                else:
                    new_image=frame[i-75:i+75,1920-1-150:1920-1,0:3]
            else:
                new_image=frame[i-75:i+75,0:150,0:3]
        elif j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[1080-1-150:1080-1,j-75:j+75,0:3]
                else:
                    new_image=frame[1080-1-150:1080-1,1920-1-150:1920-1,0:3]
        else:
            new_image=frame[1080-1-150:1080-1,0:150,0:3]
    elif j-75>=0:
                if j+75<=1920-1:
                    new_image=frame[0:150,j-75:j+75,0:3]
                else:
                    new_image=frame[0:150,1920-1-150:1920-1,0:3]
    else:
        new_image=frame[0:150,0:150,0:3]
    return new_image

#Function to calculate average value optical flow in two images and return indexes of pixels having max individual flow value
def calc_simple_flow(image1,image2):
    flow = calcOpticalFlowSF(image1, image2, layers=3, averaging_block_size=3, max_flow=4)
    n = np.sum(1 - np.isnan(flow), axis=0)
    n = np.sum(n,axis=0)
    # print(n)
    flow[np.isnan(flow)] = 0
    return np.linalg.norm(np.sum(flow, axis=(0, 1)) / n),np.unravel_index(flow.argmax(), flow.shape)[0:2],np.unravel_index(flow.argmin(), flow.shape)[0:2]

def avg_flow(image1,image2):
    flow = calcOpticalFlowSF(image1, image2, layers=3, averaging_block_size=3, max_flow=4)
    n = np.sum(1 - np.isnan(flow), axis=0)
    n = np.sum(n,axis=0)
    # print(n)
    flow[np.isnan(flow)] = 0
    return np.linalg.norm(np.sum(flow, axis=(0, 1)) / n)
#Function to generate pixel using max flow and min flow found randomly
def create_random_image_crops_pixels(frame1,frame2,random_number=10):
    max=0
    min=256
    for x in range(random_number):
        i = random.randint(0,1080)
        j=random.randint(0,1920)
        temp_image1=create_image(frame1,i,j)
        temp_image2=create_image(frame2,i,j)
        temp_flow,_,_=calc_simple_flow(temp_image1,temp_image2)
#         print(temp_flow)
        if temp_flow>max:
            i1=i
            j1=j
            max=temp_flow
        if temp_flow<min:
            i2=i
            j2=j
            min=temp_flow
#     print(i,j,max)
    return (i1,j1),(i2,j2)
def create_random_crops_based_on_Prob(frame1,frame2,random_number=10,flow_threshold = 25):
    list_of_points = []
    for x in range(random_number):
        i = random.randint(0,1080)
        j=random.randint(0,1920)
        temp_image1=create_image(frame1,i,j)
        temp_image2=create_image(frame2,i,j)
        flow = avg_flow(temp_image1,temp_image2)
        if random.random() < flow / flow_threshold:
            list_of_points.append((i,j))
            print(flow)
    return list_of_points

#(i1,j1) has pixel values for max flow and (i2,j2) for least


#Function to generate individual pixel for which flow is max or min
def create_image_crops_pixels(frame1,frame2):
    _,index_high,index_low=calc_simple_flow(frame1,frame2)
    return index_high,index_low

def save_image(frames,pixel_values,type="train",example_number=0):
    image=create_image(frames[0],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_0.jpg",image)
    image=create_image(frames[1],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_1.jpg",image)
    image=create_image(frames[2],pixel_values[0],pixel_values[1])
    cv.imwrite("./"+type+"_dataset/"+type+"_"+str(example_number)+"_2.jpg",image)
    return
def is_jumpcut(frame1, frame2, threshold=np.inf):
    """
    Detects a jumpcut between the two frames.
    :param frame1: Numpy array of the frame at time t
    :param frame2: Numpy array of the frame at time t+1
    :param threshold: Maximum difference allowed for the frames to be considered consecutive
    :return: Whether or not there is a jumpcut between the two frames
    """
    pixels_per_channel = frame1.size / 3
    hist = lambda x: np.histogram(x.reshape(-1), 8, (0, 255))[0] / pixels_per_channel
    err = lambda a, b: ((hist(a) - hist(b)) ** 2).mean()

    return err(frame1[:, :, 0], frame2[:, :, 0]) > threshold or err(frame1[:, :, 1], frame2[:, :, 1]) > threshold or err(frame1[:, :, 2], frame2[:, :, 2]) > threshold


In [69]:
example_number=0

# Read RGB image 
frame0 = cv.imread("00000.jpg")  
frame2 = cv.imread("00002.jpg")  
frame1 = cv.imread("00001.jpg")
frames=(frame0,frame1,frame2)

In [71]:
list_of_points = create_random_crops_based_on_Prob(frame0,frame2,random_number = 20)
for points in list_of_points:
    save_image(frames,points,"test",example_number)
    example_number+=1

11.182921930941662
11.097327445184314
11.069393299395426
11.051372706766896
10.246445667597587
7.093625679968066
11.048040541569002


In [ ]:
"""
START FROM HERE
"""

In [2]:
videos_path = "/home/z3u5/Desktop/dataset_videos/"
list_of_files = glob.glob(videos_path+"*.*")
print(list_of_files)

['/home/z3u5/Desktop/dataset_videos/scene9.mp4', '/home/z3u5/Desktop/dataset_videos/scene3.mp4', '/home/z3u5/Desktop/dataset_videos/scene2.mp4', '/home/z3u5/Desktop/dataset_videos/scene10.mp4', '/home/z3u5/Desktop/dataset_videos/scene1.mp4', '/home/z3u5/Desktop/dataset_videos/scene11.mp4', '/home/z3u5/Desktop/dataset_videos/redpanda.mov', '/home/z3u5/Desktop/dataset_videos/scene7.mp4', '/home/z3u5/Desktop/dataset_videos/scene4.mp4', '/home/z3u5/Desktop/dataset_videos/scene8.mp4', '/home/z3u5/Desktop/dataset_videos/deer.mp4', '/home/z3u5/Desktop/dataset_videos/peru.mp4', '/home/z3u5/Desktop/dataset_videos/scene5.mp4', '/home/z3u5/Desktop/dataset_videos/throwingfootball.mp4', '/home/z3u5/Desktop/dataset_videos/scene6.mp4']


In [13]:
list_of_files = ['/home/z3u5/Desktop/dataset_videos/mayapur.mp4']

In [ ]:
frame_capture_from_videos(list_of_files) #change paths in this function to apt paths where you want to store video frames(full)

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

...

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

In [14]:
os.mkdir("../dataset") #place where you want to store dataset images(150*150)

In [ ]:
list_of_folders = glob.glob("../training_dataset/*")#path where frames of videos are stored
print(list_of_folders)

In [ ]:
print(list_of_folders)
# list_of_folders = ['../training_dataset/frames_walkingpeople'] 

In [7]:
# example_number = 9682

In [ ]:
"""
very important following line so that our image numberings
don't clash otherwise it will be waste
"""
# example_number = 0         uncomment this line to initialize example_number to 100000
for folder in list_of_folders:
    list_of_images = sorted(glob.glob(folder+"/*.jpg"))
#     print(len(list_of_images))
    list_of_picked_images = np.random.choice(list_of_images,int(len(list_of_images)/20),replace = False)
    for image in list_of_picked_images:
        file_index  = int(os.path.basename(os.path.splitext(image)[0]))
        print(file_index)
        if not file_index+2 > len(list_of_images)-1:
            frame0 = cv.imread(list_of_images[file_index])
            frame1 = cv.imread(list_of_images[file_index+1])
            frame2 = cv.imread(list_of_images[file_index+2])
            frames=(frame0,frame1,frame2)
            if not is_jumpcut(frame0,frame2,threshold = 100000):
                list_of_points = create_random_crops_based_on_Prob(frame0,frame2)
                for points in list_of_points:
                    save_image2(frames,points,example_number)
                    example_number+=1

In [34]:
print(example_number)

9682
